In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
root='/content/drive/My Drive/Colab Notebooks/Video Summerization'
os.chdir(root)

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pickle

from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense,Dropout,GlobalAveragePooling2D,Input
%matplotlib inline

In [5]:
!pip install -r req.txt  -vvv

Streaming output truncated to the last 5000 lines.
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/dlib/external/pybind11/include/pybind11/attr.h:13,
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/dlib/external/pybind11/include/pybind11/pybind11.h:43,
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/dlib/../dlib/python/pybind_utils.h:6,
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/dlib/../dlib/python.h:6,
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/tools/python/src/opaque_types.h:6,
                   from /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/tools/python/src/decision_functions.cpp:4:
  /tmp/pip-install-ghpfbwps/dlib_7dfaa6c182b740c18a11cafa22c57cc8/dlib/external/pybind11/include/pybind11/detail/internals.h:170:61: warning: ‘int PyThread_set_key_value(int,

In [4]:
import face_recognition

In [6]:
!pip install -U pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.0.1
    Uninstalling Pillow-8.0.1:
      Successfully uninstalled Pillow-8.0.1


In [5]:
def faceRec(known_face_encodings,known_face_names,frame):
  # Initialize some variables
  face_locations = []
  face_encodings = []
  face_names = []
  process_this_frame = True

  # Grab a single frame of video

  # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
  rgb_small_frame = frame[:, :, ::-1]

  # Only process every other frame of video to save time
  if process_this_frame:
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
      # See if the face is a match for the known face(s)
      matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
      name = "Intruder"

      # Or instead, use the known face with the smallest distance to the new face
      face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      if matches[best_match_index]:
        name = known_face_names[best_match_index]

      face_names.append(name)

  process_this_frame = not process_this_frame


  # Display the results
  for (top, right, bottom, left), name in zip(face_locations, face_names):
    # Draw a box around the face
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    # Draw a label with a name below the face
    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
  return frame

In [61]:
def mobilessd(video,writer):
  protopath = 'MobileNetSSD_deploy.prototxt'
  modelpath = 'MobileNetSSD_deploy.caffemodel'

  detector = cv2.dnn.readNetFromCaffe(prototxt=protopath, caffeModel=modelpath)

  CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
            "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
            "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
            "sofa", "train", "tvmonitor"]

  while True:
      ret, frame = video.read()
      if ret is True:
        temp=frame
        frame=cv2.resize(frame,(432,243))
        (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)

        detector.setInput(blob)
        person_detections = detector.forward()

        for i in np.arange(0, person_detections.shape[2]):
            confidence = person_detections[0, 0, i, 2]
            if confidence > 0.9:
                idx = int(person_detections[0, 0, i, 1])

                if CLASSES[idx] != "person":
                    continue              
                temp=faceRec(known_face_encodings,known_face_names,temp)

                writer.write(temp)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break
      else:
        break
  return

In [62]:
def differentModel(video,writer):
  with open('model_ResNet50V2_rf' , 'rb') as f:
      model = pickle.load(f)
  count=0
  while True:
      ret,frame=video.read()
      if ret is True:
        temp=frame
        frame=cv2.resize(frame,(img_width,img_height))
        frame=frame[np.newaxis] #adding a new axis
        
        #extracting feature from image
        y=extract.predict(frame)
        pred=model.predict(y)
        print(count)
        if pred==1:
          temp=faceRec(known_face_encodings,known_face_names,temp)

          writer.write(temp)
        count+=1
      else:
        break
  return

In [6]:
# Load a sample picture and learn how to recognize it.
nihar_image = face_recognition.load_image_file("IMG_4916.jpg")
nihar_face_encoding = face_recognition.face_encodings(nihar_image)[0]

jagu_image=face_recognition.load_image_file("faceRec/jagu/IMG20211203170718.jpg")
jagu_face_encoding = face_recognition.face_encodings(jagu_image)[0]

In [7]:
known_face_encodings = [
    nihar_face_encoding,
    jagu_face_encoding
]
known_face_names = [
    "Nihar",
    "Jagannath"
]

In [36]:
base = tf.keras.applications.resnet_v2.ResNet50V2(
      include_top=False,
      weights="imagenet",
      input_shape=(243,432, 3), ##(432,243,3) for 16:9 || (480,360) for 4:3 || (640,480) for 4:3|| (960,720) for 4:3
)

94668760/94668760 [==============================] - 6s 0us/step


In [37]:
for layers in base.layers:
  layers.trainable=False

In [38]:
base.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 243, 432, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 249, 438, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 122, 216, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

In [39]:
extract=Sequential()
extract.add(base)
extract.add(Flatten())
extract.output_shape

(None, 229376)

In [63]:
# from google.colab.patches import cv2_imshow
video = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/Video Summerization/VID_20230115215252.mp4')
print (video.isOpened())

height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

fourcc=cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('test32.mp4',fourcc,30,(width,height))
print(height,width)

img_width,img_height=432,243

True
816 1072


In [64]:
# differentModel(video,writer)
mobilessd(video,writer)

In [65]:
video.release()
writer.release()
cv2.destroyAllWindows()

In [42]:
# import pickle
# with open('model_ResNet50V2_rf' , 'rb') as f:
#     model = pickle.load(f)

In [50]:
# import cv2
# from google.colab.patches import cv2_imshow

# # Load the cascade
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
# # Read the input image
# img = cv2.imread('IMG_4916.jpg')
# # Convert into grayscale
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# # Detect faces
# faces = face_cascade.detectMultiScale(gray, 1.3, 5)
# # Draw rectangle around the faces
# for (x, y, w, h) in faces:
#     cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# # Display the output
# cv2_imshow(img)